In [ ]:
import os
from PIL import Image
from tqdm import tqdm

In [ ]:
### Setting Up the Environment and Mounting Drive
def mount_drive():
  try:
      from google.colab import drive
      drive.mount('/content/drive', force_remount=True)
      COLAB = True
      print("Note: using Google CoLab")
  except:
      print("Note: not using Google CoLab")
      COLAB = False
mount_drive()

Mounted at /content/drive
Note: using Google CoLab


In [ ]:
PATH_PREFIX = "/content/drive/MyDrive/data/"
BASE_SIZE=(256,256)
TRAIN_PATH = os.path.join(PATH_PREFIX, "classifier/images/Training")
VALID_PATH = os.path.join(PATH_PREFIX, "classifier/images/Validation")
def copy_training_data(path: str = "gan/training_data/"):
  path = os.path.join(PATH_PREFIX, path)

  count = 0
  for filename in tqdm(os.listdir(path),desc ='reading images ...'):
    count += 1
    image = Image.open(path+filename)
    if image.size != BASE_SIZE:
      image = image.resize(BASE_SIZE)
    if image.mode != "RGB":
      image = image.convert("RGB")
    # print(image.size)
    if count < 1000:
      image.save(TRAIN_PATH+filename, image.format)
    elif count < 1100:
      image.save(VALID_PATH+filename, image.format)
    else:
      return

In [ ]:
copy_training_data()

reading images ...:  29%|██▉       | 1099/3748 [01:50<04:25,  9.98it/s]


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# rescale images
train_datagen = ImageDataGenerator(rescale = 1.0/255)
test_datagen  = ImageDataGenerator(rescale = 1.0/255)

# supe conviniet stuff
train_generator = train_datagen.flow_from_directory(TRAIN_PATH,
                                                    batch_size=20,
                                                    class_mode='binary',
                                                    target_size=(256, 256))

validation_generator =  test_datagen.flow_from_directory(VALID_PATH,
                                                         batch_size=20,
                                                         class_mode='binary',
                                                         target_size=(256, 256))

Found 1762 images belonging to 2 classes.
Found 131 images belonging to 2 classes.


In [ ]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    # Conv2D is the first layer of the nn
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(256, 256, 3)),
    # pooling
    tf.keras.layers.MaxPooling2D(2,2),
    # repeat
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2), 
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'), 
    tf.keras.layers.MaxPooling2D(2,2),
    # flatten the result to feed it to the dense layer
    tf.keras.layers.Flatten(), 
    # define 512 neurons for processing the output coming by the previous layers
    tf.keras.layers.Dense(512, activation='relu'), 
    # a single output neuron. The result will be 0 if the image is not ukrainian, 1 if it is ukrainian pattern
    tf.keras.layers.Dense(1, activation='sigmoid')  
])

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 254, 254, 16)      448       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 127, 127, 16)     0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 125, 125, 32)      4640      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 62, 62, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 60, 60, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 30, 30, 64)      

In [ ]:
model.compile(optimizer="adam",
              loss='binary_crossentropy',
              metrics = ['accuracy'])

In [ ]:
history = model.fit(
            train_generator, # pass in the training generator
            steps_per_epoch=50,
            epochs=15,
            validation_data=validation_generator, # pass in the validation generator
            validation_steps=50,
            verbose=2
            )

Epoch 1/15


50/50 - 42s - loss: 0.3654 - accuracy: 0.8340 - val_loss: 0.4108 - val_accuracy: 0.9084 - 42s/epoch - 834ms/step
Epoch 2/15
50/50 - 41s - loss: 0.2784 - accuracy: 0.8860 - 41s/epoch - 820ms/step
Epoch 3/15
50/50 - 41s - loss: 0.2771 - accuracy: 0.8800 - 41s/epoch - 816ms/step
Epoch 4/15
50/50 - 41s - loss: 0.2026 - accuracy: 0.9120 - 41s/epoch - 821ms/step
Epoch 5/15
50/50 - 41s - loss: 0.2154 - accuracy: 0.9100 - 41s/epoch - 812ms/step
Epoch 6/15
50/50 - 41s - loss: 0.1659 - accuracy: 0.9330 - 41s/epoch - 817ms/step
Epoch 7/15
50/50 - 40s - loss: 0.1289 - accuracy: 0.9530 - 40s/epoch - 805ms/step
Epoch 8/15
50/50 - 41s - loss: 0.1171 - accuracy: 0.9660 - 41s/epoch - 825ms/step
Epoch 9/15
50/50 - 40s - loss: 0.0866 - accuracy: 0.9633 - 40s/epoch - 794ms/step
Epoch 10/15
50/50 - 41s - loss: 0.0580 - accuracy: 0.9770 - 41s/epoch - 811ms/step
Epoch 11/15
50/50 - 41s - loss: 0.0463 - accuracy: 0.9800 - 41s/epoch - 828ms/step
Epoch 12/15
50/50 - 40s - loss: 0.0324 - accuracy: 0.9870 - 40s/e

In [ ]:
model.save(os.path.join(PATH_PREFIX, 'classifier/model/my_model'))

In [ ]:

import numpy as np
import tensorflow as tf
import os
from PIL import Image
import numpy as np
from tqdm import tqdm
mount_drive()
PATH_PREFIX = "/content/drive/MyDrive/data/"

# if testing on outside data we need to check that image size is 256 by 256
def test_prediction(test_img_path: str = 'gan/training_data/train_img_3781.jpeg'):
  model = tf.keras.models.load_model(os.path.join(PATH_PREFIX, 'classifier/model/my_model'))
  # prediction on the uploaded image
  path = os.path.join(PATH_PREFIX, test_img_path) # load the image on Colab
  img = Image.open(path)
  if img.size != (256,256):
    print('Please reformat image to be 256x256')
  # normalize
  x = np.asarray(np.copy(img))
  x = x/255 
  x = np.expand_dims(x, axis=0)
  # flatten the output
  images = np.vstack([x])

  # prediction!
  classes = model.predict(images, batch_size=10)

  print(classes[0])

  if classes[0] > 0.5:
    print("It's ukrainian embroidery")
  else:
    print("It's not ukrainian embroidery")

def test_multiple_images(folder_path: str = "/content/drive/MyDrive/data/gan/fakes/") -> None:
  for filename in tqdm(os.listdir(folder_path),desc ='reading images ...'):
    test_prediction(folder_path+filename)
# test_multiple_images()
test_multiple_images("/content/drive/MyDrive/data/classifier/images/Validation/Non_Ukrainian/")

Mounted at /content/drive
Note: using Google CoLab


reading images ...:   0%|          | 0/31 [00:00<?, ?it/s]

1/1 [==============================] - 0s 108ms/step


reading images ...:   3%|▎         | 1/31 [00:01<00:55,  1.85s/it]

[0.00057126]
It's not ukrainian embroidery
1/1 [==============================] - 0s 106ms/step


...

[0.00221423]
It's not ukrainian embroidery
1/1 [==============================] - 0s 107ms/step


reading images ...:  97%|█████████▋| 30/31 [00:57<00:01,  1.89s/it]

[1.0320348e-07]
It's not ukrainian embroidery
1/1 [==============================] - 0s 102ms/step


reading images ...: 100%|██████████| 31/31 [00:59<00:00,  1.90s/it]

[3.2422392e-06]
It's not ukrainian embroidery
